In [1]:
from rbc.mapd import RemoteMapD

In [2]:
omnisci = RemoteMapD(use_host_target=True)  # RemoteMapD(user=, password=, host=, port=, dbname=)

In [3]:
omnisci.sql_execute('drop table if exists mytable')
omnisci.sql_execute('create table if not exists mytable (i INT[]);');
for i in range(5):
    omnisci.sql_execute('insert into mytable values (ARRAY[1,2,3,'+str(i)+'])');
list(omnisci.sql_execute('select * from mytable')[1])

[([1, 2, 3, 0],),
 ([1, 2, 3, 1],),
 ([1, 2, 3, 2],),
 ([1, 2, 3, 3],),
 ([1, 2, 3, 4],)]

In [4]:
@omnisci('i32(i32[])')
def sum_i32(x):
    s = 0
    for i in range(len(x)):
        s += x[i]
    return s

In [5]:
descr, result = omnisci.sql_execute(
    'select i, sum_i32(i) from mytable')

list(result)

[([1, 2, 3, 0], 6),
 ([1, 2, 3, 1], 7),
 ([1, 2, 3, 2], 8),
 ([1, 2, 3, 3], 9),
 ([1, 2, 3, 4], 10)]

In [6]:
print(sum_i32)

; ModuleID = 'rbc.irtools.compile_to_IR'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@"_ZN08NumbaEnv8__main__11sum_i32$245E22$28STRUCT__iPLbK$29$2a" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv3rbc13omnisci_array14mapd_array_len12$3clocals$3e16$3clambda$3e$242E22$28STRUCT__iPLbK$29$2a" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv3rbc13omnisci_array18mapd_array_getitem12$3clocals$3e16$3clambda$3e$243E22$28STRUCT__iPLbK$29$2ax" = common local_unnamed_addr global i8* null

; Function Attrs: norecurse nounwind readonly
define i32 @sum_i32({ i32*, i64, i8 }* nocapture readonly %.1) local_unnamed_addr #0 {
entry:
  %.17.i.i = getelementptr { i32*, i64, i8 }, { i32*, i64, i8 }* %.1, i64 0, i32 1
  %.18.i.i = load i64, i64* %.17.i.i, align 8, !noalias !0
  %.95.i = icmp sgt i64 %.18.i.i, 0
  br i1 %.95.i, label %B20.endif.lr.ph.i, label %"_ZN8__main__11sum_i32$245E22$28STRUCT_